# Dependencies

In [ ]:
#%pip install pyautogui opencv-python easyocr

In [ ]:
#%pip install torch==2.3.1 torchaudio==2.3.1 torchvision==0.18.1

# Libraries

In [ ]:
import os
import cv2
import time
import easyocr
import logging
import pyautogui
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

In [ ]:
start_time = time.time()

In [ ]:
def take_screenshot():
    screenshot = pyautogui.screenshot()
    return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

def get_colors():
    return np.array([
        [247, 236, 227], # White
        [158, 92, 21],   # Blue
        [200, 113, 27],  # Background
    ], dtype=np.uint8)

def create_mask_colors(screenshot, colors):
    return [cv2.inRange(screenshot, color, color) for color in colors]

def get_contours(masks):
    combined_mask = np.bitwise_or.reduce(masks)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_dimensions(contours):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    return x, y, w, h

In [ ]:
screenshot = take_screenshot()
colors = get_colors()
masks = create_mask_colors(screenshot, [colors[-1]])
colors = colors[:-1]

table_contours = get_contours(masks)
table_dims = get_dimensions(table_contours)

x, y, w, h = table_dims
square_masks = create_mask_colors(screenshot[y:y+h, x:x+w], colors)

square_contours_white = get_contours([square_masks[0]])
square_dims_white = get_dimensions(square_contours_white)

square_contours_blue = get_contours([square_masks[1]])
square_dims_blue = get_dimensions(square_contours_blue)

index = 0
square_distance = max(square_dims_white[index], square_dims_blue[index])
square_size = max(np.concatenate((square_dims_white[2:], square_dims_blue[2:])))

border_size, line_size = 12, 4
square_dims = (square_size, border_size, line_size)

table_dims, square_dims

In [ ]:
x, y, w, h = table_dims
plt.imshow(cv2.cvtColor(screenshot[y:y+h, x:x+w], cv2.COLOR_BGR2RGB), cmap='gray')
plt.axis('off')  # Ocultar los ejes
plt.show()

In [ ]:
logging.getLogger().setLevel(logging.ERROR)
reader = easyocr.Reader(['en'])

In [ ]:
def get_square(screenshot, table_dims, i, j, square_size):
    square_size, border_size, line_size = square_dims
    
    sx = j * (square_size + line_size * int(j > 0)) + border_size
    sy = i * (square_size + line_size * int(i > 0)) + border_size
    
    x, y, w, h = (
        table_dims[0] + sx, 
        table_dims[1] + sy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    if original_width == width:
        return square
    
    ratio = width / float(original_width)
    height = int(original_height * ratio)
    
    return cv2.resize(square, (width, height), interpolation=inter)

def str_to_number(str_value):
    try:
        return int(str_value)
    except ValueError:
        return

def get_number(square):
    results = reader.readtext(square)
    if not results:
        return -1

    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return str_to_number(sorted_results[0][1])
    
def get_cell_value(square, color):
    mask = cv2.inRange(square, color, color)
    color_percentage = (np.sum(mask) / (mask.size * 255)) * 100

    if color_percentage == 0:
        return 0

    return get_number(square)

In [ ]:
_, _, width, height = table_dims
square_size, border_size, line_size = square_dims

n = ((height - 2 * border_size + line_size) // (square_size + line_size))
m = ((width - 2 * border_size + line_size) // (square_size + line_size))

n, m

In [ ]:
table = np.zeros((n, m))

for i in range(len(table)):
    for j in range(len(table[i])):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 80)
        table[i, j] = get_cell_value(resized_square, colors[1])
    print(f"Fila {i + 1} de {len(table)} leida")

In [ ]:
def show_table(table):
    str_value = ""
    for row in table:
        for col in row:
            num = float(col)
            value = f" {num:.0f}" if num < 0 else f"  {num:.0f}"
            str_value += value + ""
        str_value += "\n"

    print(str_value)

In [ ]:
show_table(table)

In [ ]:
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    return -3

def get_adjacent_cells(table, i, j):
    adjacent_cells = []
    
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell_value = get_cell(table, ni, nj)
        
        if cell_value != -3:
            adjacent_cells.append((ni, nj, cell_value))
    
    return adjacent_cells

def adjacent_cell_with_positive_numbers(table, i, j):
    return [(ni, nj, cell_value) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0]

def num_adjacent_cell_with_positive_numbers(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0)

def mines_around_cell(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == -2)

def empty_cells_around(table, i, j):
    return {(ni, nj) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == 0}

def is_special_case(table, i, j):
    cases = [
        [ # Arriba-Derecha
            [(0, -2), (1, -2), (2, -2), (2, -1), (2, 0)], # Bloqueadas
            [(0, -1), (1, 0)], # Diagonales con numeros
            (1, -1) # Diagonal con un 1
        ],
        [ # Derecha-Abajo
            [(2, 0), (2, 1), (2, 2), (1, 2), (0, 2)], # Bloqueadas
            [(1, 0), (0, 1)], # Diagonales con numeros
            (1, 1) # Diagonal con un 1
        ],
        [ # Abajo-Izquierda
            [(0, 2), (-1, 2), (-2, 2), (-2, 1), (-2, 0)], # Bloqueadas
            [(0, 1), (-1, 0)], # Diagonales con numeros
            (-1, 1) # Diagonal con un 1
        ],
        [ # Izquierda-Arriba
            [(-2, 0), (-2, -1), (-2, -2), (-1, -2), (0, -2)], # Bloqueadas
            [(-1, 0), (0, -1)], # Diagonales con numeros
            (-1, -1) # Diagonal con un 1
        ]
    ]

    if get_cell(table, i, j) == 0:
        for case in cases:
            num_lockeds = sum(1 for dx, dy in case[0] if get_cell(table, i + dy, j + dx) != 0)
            is_numbers = all(get_cell(table, i + dy, j + dx) > 0 for dx, dy in case[1])
            
            dx, dy = case[2]
            is_one = get_cell(table, i + dy, j + dx) == 1

            # Caso 1
            if num_lockeds >= 3 and is_numbers and is_one:
                return True

            is_ones = all(get_cell(table, i + dy, j + dx) == 1 for dx, dy in case[1])
            num_lockeds = sum(1 for dx, dy in case[0] if get_cell(table, i + dy, j + dx) != 0)
            
            # Caso 2
            if is_one and is_ones and num_lockeds > 2:
                return True

    return False

def is_valid_special_case(table, i, j):
    if is_special_case(table, i, j):
        new_table = [row.copy() for row in table]
        new_table[i][j] = -2
        return is_valid_board(new_table)
    return False

def heuristic(table, i, j):
    mines_around, empty_cells = 0, set()
    adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)

    if is_valid_special_case(table, i, j):
        return 100
    
    if not adjacent_cells or get_cell(table, i, j) != 0:
        return 0

    for ni, nj, cell_value in adjacent_cells:
        mines_around += cell_value - mines_around_cell(table, ni, nj)
        empty_cells.update(empty_cells_around(table, ni, nj))

    if len(empty_cells) == 1 and mines_around > 0:
        return 100
        
    return mines_around / len(empty_cells) if empty_cells else int(mines_around > 0)

In [ ]:
class TreeNode:
    def __init__(self, table, movement=None):
        self.table = table
        self.children = []
        self.movement = movement

    def find_paths(self):
        def dfs(node, leaves):
            if node is None:
                return

            if not node.children:
                leaves.append(node.table)

            for child in node.children:
                dfs(child, leaves)

        leaves = []
        dfs(self, leaves)
        return leaves

In [ ]:
def calculate_heuristics(table):
    heuristics = []
    for i in range(len(table)):
        for j in range(len(table[i])):
            value = heuristic(table, i, j)
            if value > 0:
                heuristics.append((i, j, value))
    return sorted(heuristics, key=lambda x: x[2])

def is_valid_board(table, full=False):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell_value = get_cell(table, i, j)
            mines_around = mines_around_cell(table, i, j)
            adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)
            cond = (cell_value < mines_around) or (mines_around < cell_value and full)

            if adjacent_cells and cell_value > 0 and cond:
                return False
    return True

def can_continue(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            if 0 < cell and mines_around_cell(table, i, j) < cell:
                return True
    return False

def calculate_probability(matrices, target_number, row_index, col_index):
    total_matrices = len(matrices)
    count = sum(matrix[row_index][col_index] == target_number for matrix in matrices)
    return 0 if total_matrices == 0 else (count / total_matrices), count

def get_probabilities(table, paths, n, m):
    prob_table = np.ones((n, m))
    
    for i in range(n):
        for j in range(m):
            prob, count = calculate_probability(paths, -2, i, j)
            num_adj = num_adjacent_cell_with_positive_numbers(table, i, j)
            cell = get_cell(table, i, j)
            
            if 0 < count or (cell == 0 and 0 < num_adj):
                prob_table[i, j] = prob
            else:
                prob_table[i, j] = 1
                
    return prob_table

def find_min_position(matrix):
    np_matrix = np.array(matrix)
    return np.unravel_index(np.argmin(np_matrix), np_matrix.shape)

def find_best_movement(table, root):
    paths = root.find_paths()
    valid_paths = [path for path in paths if is_valid_board(path, full=True)]
    print(len(paths), len(valid_paths))
    prob_table = get_probabilities(table, valid_paths, len(table), len(table[0]))
    return find_min_position(prob_table)

def table_to_tuple(table):
    return tuple(tuple(row) for row in table)

def is_valid_possibility(root, visited_tables, i, j):
    new_table = [row.copy() for row in root.table]
    new_table[i][j] = -2
    table_tuple = table_to_tuple(new_table)
    return is_valid_board(new_table) and table_tuple not in visited_tables

def set_possibility(root, visited_tables, i, j):
    new_table = [row.copy() for row in root.table]
    new_table[i][j] = -2
    
    table_tuple = table_to_tuple(new_table)
    visited_tables.add(table_tuple)
    
    node = TreeNode(new_table, (i, j))
    root.children.append(node)
    
    return node

def generate_possibilities(root, visited_tables=set(), invalid_movements=[]):
    heuristics_list = calculate_heuristics(root.table)
    valid_heuristics_list = [h for h in heuristics_list if (h[0], h[1]) not in invalid_movements]

    if not can_continue(root.table) or not valid_heuristics_list:
        return is_valid_board(root.table, full=True)

    land_mine = valid_heuristics_list[-1]
    i, j, h = land_mine
    if h == 100:
        node = set_possibility(root, visited_tables, i, j)
        return generate_possibilities(node, visited_tables, invalid_movements)

    valid_movements, new_invalid_movements = [], []
    while len(valid_heuristics_list) > 0:
        i, j, _ = valid_heuristics_list.pop()
        if is_valid_possibility(root, visited_tables, i, j):
            valid_movements.append((i, j))
        else:
            new_invalid_movements.append((i, j))
    
    if not valid_movements:
        return False

    all_invalid_movements = invalid_movements + new_invalid_movements
    count = len(valid_movements)
    while len(valid_movements) > 0:
        i, j = valid_movements.pop()
        node = set_possibility(root, visited_tables, i, j)
        
        if not generate_possibilities(node, visited_tables, all_invalid_movements):
            root.children = [child for child in root.children if child.movement not in all_invalid_movements]
            count -= 1
    
    return count > 0

In [ ]:
rootNode = TreeNode(table)
generate_possibilities(rootNode)

In [ ]:
square_i, square_j = find_best_movement(table, rootNode)

In [ ]:
def show_best_movement(table_dims, i, j, square_dims):
    x, y, w, h = table_dims
    square_size, border_size, line_size = square_dims
    
    sx = j * (square_size + line_size * int(j > 0)) + border_size
    sy = i * (square_size + line_size * int(i > 0)) + border_size
        
    table = screenshot[y:y+h, x:x+w]    
    cv2.rectangle(table, (sx, sy), (sx+square_size, sy+square_size), (0, 0, 0), 2)

    plt.imshow(cv2.cvtColor(table, cv2.COLOR_BGR2RGB), cmap='gray')
    plt.axis('off')  # Ocultar los ejes
    plt.show()

In [ ]:
show_best_movement(table_dims, square_i, square_j, square_dims)

In [ ]:
end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time

minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"{minutes}min {seconds}s")